In [16]:
import org.apache.spark.sql.SQLContext
import org.apache.spark.sql.types.{StructType, StructField, StringType, DoubleType};
import com.databricks.spark.xml._

val sqlContext = new SQLContext(sc)
import sqlContext.implicits._
sqlContext.sql("set spark.sql.caseSensitive=true") 
spark.sparkContext.uiWebUrl

sqlContext = org.apache.spark.sql.SQLContext@3811830e


lastException: Throwable = null


Some(http://iuni2:4043)

In [17]:
//val sqlContext = new SQLContext(sc)
val df = sqlContext.read
  .format("com.databricks.spark.xml")
  .option("rowTag", "REC")
  //.schema(customSchema)
  .load("/WoSraw/xmlRaw/xmlRaw/*.xml.gz")

df.show(2)
df.printSchema()

+-------------------+--------------------+---------+---------+-------------+---------+------------+--------------------+--------+----+--------+---------+-----------------+--------------------+----+
|                UID|    _r_id_disclaimer|abstracts|addresses|category_info|citedWork|contributors|        dynamic_data|fund_ack|item|keywords|reference|reprint_addresses|         static_data| sup|
+-------------------+--------------------+---------+---------+-------------+---------+------------+--------------------+--------+----+--------+---------+-----------------+--------------------+----+
|WOS:000201885200002|ResearcherID data...|     null|     null|         null|     null|        null|[[[[[, accession_...|    null|null|    null|     null|             null|[,,,,, [, [, 0,],...|null|
|WOS:000200400400015|ResearcherID data...|     null|     null|         null|     null|        null|[[[[[, accession_...|    null|null|    null|     null|             null|[,,,,, [, [, 0,],...|null|
+---------

df = [UID: string, _r_id_disclaimer: string ... 13 more fields]


[UID: string, _r_id_disclaimer: string ... 13 more fields]

In [18]:
import org.apache.spark.sql.SaveMode
df.write.mode(SaveMode.Overwrite)
.parquet("/WoSraw/parquet")
//WoS2.describe().filter($"summary" === "count").show

In [66]:
val WoS = spark.read.format("parquet").load("/WoSraw/parquet/*.parquet")

val WoS2 = WoS.select("UID","dynamic_data.cluster_related.identifiers.identifier","static_data.summary.doctypes.doctype",
                      "static_data.summary.names.name","static_data.summary.titles","static_data.summary.conferences.conference",
                      "static_data.fullrecord_metadata.addresses.address_name","static_data.fullrecord_metadata.abstracts.abstract.abstract_text",
                      "static_data.fullrecord_metadata.keywords.keyword",
                      "static_data.fullrecord_metadata.references.reference",
                      "static_data.summary.pub_info","static_data.summary.publishers","static_data.fullrecord_metadata.fund_ack.grants.grant")
WoS2.printSchema

root
 |-- UID: string (nullable = true)
 |-- identifier: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- _VALUE: string (nullable = true)
 |    |    |-- _type: string (nullable = true)
 |    |    |-- _value: string (nullable = true)
 |-- doctype: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- name: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- _addr_no: string (nullable = true)
 |    |    |-- _orcid_id_tr: string (nullable = true)
 |    |    |-- _r_id_tr: string (nullable = true)
 |    |    |-- _reprint: string (nullable = true)
 |    |    |-- _role: string (nullable = true)
 |    |    |-- _seq_no: long (nullable = true)
 |    |    |-- display_name: string (nullable = true)
 |    |    |-- email_addr: string (nullable = true)
 |    |    |-- first_name: string (nullable = true)
 |    |    |-- full_name: string (nullable = true)
 |    |    |-- last_name: string (nullable = true)
 

WoS = [UID: string, _r_id_disclaimer: string ... 13 more fields]
WoS2 = [UID: string, identifier: array<struct<_VALUE:string,_type:string,_value:string>> ... 11 more fields]


[UID: string, identifier: array<struct<_VALUE:string,_type:string,_value:string>> ... 11 more fields]

In [64]:
WoS2.write.mode(SaveMode.Overwrite).json("/WoSraw/json")

In [61]:
WoS2.show(10, false)

+-------------------+------------------------------------------------------------------------------------------------+------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------+-------+---------------------------

In [62]:
import org.apache.spark.sql.{Row, Column, SQLContext}
import org.apache.spark.sql.functions._
//WoS2.filter(WoS2("conference").isNull).count()
//WoS2.filter((WoS2($"conference") === "") || WoS2($"conference").isNull || WoS2($"conference").isNaN).count()
WoS2.select(WoS2.columns.map(colName => {
    count(when(col(colName).isNull, true)) as s"${colName}_nulls_count"
  }): _*)
.show(10)

+---------------+----------------------+-------------------+----------------+------------------+----------------------+------------------------+-------------------------+-------------------+---------------+--------------------+----------------------+-----------------+
|UID_nulls_count|identifier_nulls_count|doctype_nulls_count|name_nulls_count|titles_nulls_count|conference_nulls_count|address_name_nulls_count|abstract_text_nulls_count|keyword_nulls_count|uid_nulls_count|pub_info_nulls_count|publishers_nulls_count|grant_nulls_count|
+---------------+----------------------+-------------------+----------------+------------------+----------------------+------------------------+-------------------------+-------------------+---------------+--------------------+----------------------+-----------------+
|              0|                401070|                  0|               0|                 0|              62340301|                22434779|                 38504474|           50059771|   

In [30]:
val docTypeCount = WoS2.groupBy("doctype").count()
docTypeCount.show(docTypeCount.count.toInt, false)

+---------------------------------------------------+--------+
|doctype                                            |count   |
+---------------------------------------------------+--------+
|[Software Review, Book Chapter]                    |1       |
|[Script]                                           |2789    |
|[Editorial Material, Book Chapter]                 |170877  |
|[Biographical-Item]                                |128018  |
|[Abstract of Published Item]                       |13433   |
|[Article, Book Chapter]                            |1140635 |
|[Correction, Addition]                             |146952  |
|[Discussion]                                       |80417   |
|[Bibliography]                                     |12870   |
|[Review]                                           |1776398 |
|[Correction, Retracted Publication]                |62      |
|[Item About an Individual]                         |88287   |
|[Article, Retracted Publication]                   |45

docTypeCount = [doctype: array<string>, count: bigint]


[doctype: array<string>, count: bigint]

In [25]:
WoS2.describe().filter($"summary" === "count").show

+-------+--------+---------+
|summary|     UID|citedWork|
+-------+--------+---------+
|  count|73145376|    33023|
+-------+--------+---------+



In [13]:
val confCount = WoS2.filter($"conf_titles".isNotNull)
confCount.describe().filter($"summary" === "count").show
val grantCount = WoS2.filter($"grant_agency".isNotNull)
grantCount.describe().filter($"summary" === "count").show
//WoS2.describe().filter($"summary" === "count").show

lastException = null


Name: org.apache.spark.SparkException
Message: Job aborted due to stage failure: 
Aborting TaskSet 11.0 because task 892 (partition 892)
cannot run anywhere due to node and executor blacklist.
Most recent failure:
Lost task 892.0 in stage 11.0 (TID 4306, iuni5, executor 80): ExecutorLostFailure (executor 80 exited caused by one of the running tasks) Reason: Container killed by YARN for exceeding memory limits.  16.0 GB of 15.5 GB physical memory used. Consider boosting spark.yarn.executor.memoryOverhead or disabling yarn.nodemanager.vmem-check-enabled because of YARN-4714.

Blacklisting behavior can be configured via spark.blacklist.*.

StackTrace: Aborting TaskSet 11.0 because task 892 (partition 892)
cannot run anywhere due to node and executor blacklist.
Most recent failure:
Lost task 892.0 in stage 11.0 (TID 4306, iuni5, executor 80): ExecutorLostFailure (executor 80 exited caused by one of the running tasks) Reason: Container killed by YARN for exceeding memory limits.  16.0 GB of

In [102]:
val edges = WoS2.select("UID","pub_info._pubyear","reference.uid").withColumn("cited", explode($"uid"))
//edges.show(false)
edges.describe().filter($"summary" === "count").show

+-------+----------+----------+----------+
|summary|       UID|  _pubyear|     cited|
+-------+----------+----------+----------+
|  count|1474095899|1474095899|1472793641|
+-------+----------+----------+----------+



edges = [UID: string, _pubyear: bigint ... 2 more fields]


[UID: string, _pubyear: bigint ... 2 more fields]

In [68]:
val PMID = spark.read.format("csv").option("delimiter", "|").option("header", true).load("/WoSraw/PMID_UT_File.txt")
PMID.show

+-------------------+---------------+
|             FACTID|           PMID|
+-------------------+---------------+
|WOS:A1976CP17000008|MEDLINE:1000001|
|WOS:A1976CP17000010|MEDLINE:1000003|
|WOS:A1976CP17000012|MEDLINE:1000005|
|WOS:A1976CP17000014|MEDLINE:1000007|
|WOS:A1976CP17000016|MEDLINE:1000009|
|WOS:A1978FS77900015| MEDLINE:100001|
|WOS:A1976CP17000018|MEDLINE:1000010|
|WOS:A1976CR07200001|MEDLINE:1000012|
|WOS:A1976CR07200003|MEDLINE:1000014|
|WOS:A1976CM49800001|MEDLINE:1000016|
|WOS:A1976CM49800010|MEDLINE:1000018|
|WOS:A1976BQ50600001|MEDLINE:1000021|
|WOS:A1976BQ50600006|MEDLINE:1000023|
|WOS:A1976BY10700001|MEDLINE:1000025|
|WOS:A1976BY10700006|MEDLINE:1000027|
|WOS:A1976BY10700010|MEDLINE:1000029|
|WOS:A1978FH04500020| MEDLINE:100003|
|WOS:A1976BY10700011|MEDLINE:1000030|
|WOS:A1976CJ83400004|MEDLINE:1000032|
|WOS:A1976CJ83400012|MEDLINE:1000034|
+-------------------+---------------+
only showing top 20 rows



PMID = [FACTID: string, PMID: string]


[FACTID: string, PMID: string]

In [69]:
PMID.describe().filter($"summary" === "count").show

+-------+--------+--------+
|summary|  FACTID|    PMID|
+-------+--------+--------+
|  count|20993269|20993269|
+-------+--------+--------+



In [72]:
val matchedPapers = PMID.join(WoS2, PMID("FACTID")===WoS2("UID"), "inner")
matchedPapers.describe().filter($"summary" === "count").show

+-------+--------+--------+--------+
|summary|  FACTID|    PMID|     UID|
+-------+--------+--------+--------+
|  count|20993085|20993085|20993085|
+-------+--------+--------+--------+



In [98]:
val matchedPaperID = matchedPapers.select("PMID","UID","reference.uid","reference.year").toDF("medID","UID","referenceID","referenceYear")
val matchedPaperRefs = matchedPaperID.select(substring(col("medID"),9,20).as("PID"), col("referenceID"), col("referenceYear"))
matchedPaperRefs.show

+--------+--------------------+--------------------+
|     PID|         referenceID|       referenceYear|
+--------+--------------------+--------------------+
| 9519093|[WOS:A1995QH39300...|[1995, 1982, 1993...|
| 8958073|[WOS:A19619006A00...|[1961, 1989, 1988...|
| 8958094|[WOS:000070949000...|[1994, 1993, 1983...|
| 8906151|[WOS:A1991GU23700...|[1991, 1995, 1994...|
| 8906195|[WOS:A1989AY30100...|[1989, 1968, 1994...|
|19909325|[WOS:A1991EX60500...|[1991, 1995, 1994...|
| 9463630|[WOS:A1995TA12500...|[1995, 1985, 1995...|
| 9502174|[WOS:A1991GL51500...|[1991, 1991, 1991...|
| 9409391|                null|                null|
| 9414711|[WOS:A1997XB70300...|[1997, 1996, 1988...|
| 9414816|[WOS:A1990CP23800...|[1990, 1996, 1997...|
| 9414875|[WOS:000070963500...|[1996, 1982, 1994...|
| 8958912|[WOS:A1994PX14600...|[1994, 1995, 1992...|
| 8958947|[WOS:A1994PK12600...|        [1994, 1994]|
| 8958961|[WOS:A1995QV29000...|[1995, 1994, 1994...|
| 8958967|[WOS:A1995QW90100...|[1995, 1994, 19

matchedPaperID = [medID: string, UID: string ... 2 more fields]
matchedPaperRefs = [PID: string, referenceID: array<string> ... 1 more field]


lastException: Throwable = null


[PID: string, referenceID: array<string> ... 1 more field]

In [109]:
val matchedCitaions = PMID.join(edges, PMID("FACTID")===edges("cited"), "inner")
val matchedCites =  matchedCitaions.select(substring(col("PMID"),9,20).as("PID"), col("_pubyear"), col("cited"),col("UID").as("citing"))
matchedCites.describe().filter($"summary" === "count").show

+-------+---------+---------+---------+---------+
|summary|      PID| _pubyear|    cited|   citing|
+-------+---------+---------+---------+---------+
|  count|576612422|576612422|576612422|576612422|
+-------+---------+---------+---------+---------+



matchedCitaions = [FACTID: string, PMID: string ... 4 more fields]
matchedCites = [PID: string, _pubyear: bigint ... 2 more fields]


[PID: string, _pubyear: bigint ... 2 more fields]

In [110]:
matchedCites.show(false)

+-------+--------+-------------------+-------------------+
|PID    |_pubyear|cited              |citing             |
+-------+--------+-------------------+-------------------+
|9519093|2000    |WOS:000070921600010|WOS:000166622100003|
|9519093|2009    |WOS:000070921600010|WOS:000266298100010|
|9519093|2007    |WOS:000070921600010|WOS:000246193200004|
|9519093|2009    |WOS:000070921600010|WOS:000265424000003|
|9519093|2014    |WOS:000070921600010|WOS:000329866300095|
|9519093|2003    |WOS:000070921600010|WOS:000183209700016|
|9519093|2009    |WOS:000070921600010|WOS:000265569200013|
|9519093|2012    |WOS:000070921600010|WOS:000305931800009|
|9519093|2012    |WOS:000070921600010|WOS:000213680400010|
|9519093|2010    |WOS:000070921600010|WOS:000289422600002|
|9519093|2002    |WOS:000070921600010|WOS:000178417100002|
|9519093|2008    |WOS:000070921600010|WOS:000260697300011|
|9519093|2017    |WOS:000070921600010|WOS:000391034600008|
|9519093|2004    |WOS:000070921600010|WOS:00022475800000

In [75]:
val PMpapers = spark.read.format("csv").option("header", true).load("/data1/pubMedList.csv")
PMpapers.show

+--------+--------------------+--------------------+--------------------+-------+--------+--------------------+
|    PMID|               Title|              Author|             Journal|Pubyear|Category|             Details|
+--------+--------------------+--------------------+--------------------+-------+--------+--------------------+
|22283023|[Second meeting o...|Sistac Ballarín J...|Rev Esp Anestesio...|   2010|medicine|Rev Esp Anestesio...|
|22283018|[Bringing clinica...|Sancho R, Rábago ...|Rev Esp Anestesio...|   2010|medicine|Rev Esp Anestesio...|
|22145552|The Mental Health...|    Glover-Thomas N.|             Med Law|   2010|medicine|Med Law. 2010 Dec...|
|22106586|Postgraduate surg...|           Tandon A.| Indian J Med Ethics|   2010|medicine|Indian J Med Ethi...|
|22046717|[Diagnostic use o...|          Talbot JN.|  Bull Acad Natl Med|   2010|medicine|Bull Acad Natl Me...|
|22046713|[Advances treatme...|            Orcel P.|  Bull Acad Natl Med|   2010|medicine|Bull Acad Natl

PMpapers = [PMID: string, Title: string ... 5 more fields]


lastException: Throwable = null


[PMID: string, Title: string ... 5 more fields]

In [113]:
PMpapers.describe().filter($"summary" === "count").show

+-------+------+------+------+-------+-------+--------+-------+
|summary|  PMID| Title|Author|Journal|Pubyear|Category|Details|
+-------+------+------+------+-------+-------+--------+-------+
|  count|836628|836628|836628| 836628| 836628|  836628| 836628|
+-------+------+------+------+-------+-------+--------+-------+



In [112]:
val matched = PMpapers.join(matchedCites, PMpapers("PMID")===matchedCites("PID"), "left")
//matched.show
matched.describe().filter($"summary" === "count").show

+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
|summary|    PMID|   Title|  Author| Journal| Pubyear|Category| Details|     PID|_pubyear|   cited|  citing|
+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
|  count|15210460|15210460|15210460|15210460|15210460|15210460|15210460|14786288|14786288|14786288|14786288|
+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+



matched = [PMID: string, Title: string ... 9 more fields]


lastException: Throwable = null


[PMID: string, Title: string ... 9 more fields]

In [116]:
val matchedRefs = matched.orderBy("_pubyear").groupBy("PMID").agg(concat_ws("|", collect_list("_pubyear")) as "citedYears")
//matchedPapers.describe().filter($"summary" === "count").show
matchedRefs.show

+--------+--------------------+
|    PMID|          citedYears|
+--------+--------------------+
|10024099|2012|2013|2010|20...|
| 1003397|                1980|
|10036666|                2004|
| 1004266|                    |
|10066299|                    |
|10069318|                    |
|10072198|                    |
|10078382|                2005|
|10087323|2012|2003|2000|20...|
|10100247|                    |
|10103660|                    |
|10105900|                    |
|10105908|                    |
|10110019|                    |
|10116443|                    |
|10118508|                    |
|10126143|                    |
|10133958|                    |
|10134612|                    |
|10136192|                    |
+--------+--------------------+
only showing top 20 rows



matchedRefs = [PMID: string, citedYears: string]


[PMID: string, citedYears: string]

In [122]:
val matchedRefs2 = matchedRefs.toDF("PID","citedYears")
val matched2 = PMpapers.join(matchedRefs2, PMpapers("PMID")===matchedRefs2("PID"), "inner")
//matched.show
matched2.describe().filter($"summary" === "count").show

+-------+------+------+------+-------+-------+--------+-------+------+----------+
|summary|  PMID| Title|Author|Journal|Pubyear|Category|Details|   PID|citedYears|
+-------+------+------+------+-------+-------+--------+-------+------+----------+
|  count|836628|836628|836628| 836628| 836628|  836628| 836628|836628|    836628|
+-------+------+------+------+-------+-------+--------+-------+------+----------+



matchedRefs2 = [PID: string, citedYears: string]
matched2 = [PMID: string, Title: string ... 7 more fields]


[PMID: string, Title: string ... 7 more fields]

In [123]:
matched2.coalesce(1).write.mode(SaveMode.Overwrite)
    .format("com.databricks.spark.csv")
    .option("header", "true")
    .save("/data1/pubMedListRef.csv")